In [4]:
%%capture
#!pip install keras==2.1.2 --upgrade

# imports
from pathlib import Path
import getpass
from time import time
import numpy as np
import os 
import csv
import tensorflow as tf
import keras
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import regularizers
from keras.models import Model
from keras.optimizers import SGD, Adamax
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint    

In [5]:
# commands
#!ls 
#!kill -9 -1
#!ps aux | grep tensorboard
#!kill -9 169 

In [6]:
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wp6LdtNxazu7MiLoSp0cbU4hrGu1v7sg' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1wp6LdtNxazu7MiLoSp0cbU4hrGu1v7sg" -O Resized_180.tar && rm -rf /tmp/cookies.txt

In [12]:
# configure setup
batch_size = 16
classes = 12
dropout = 0.25
kernel_reg = 0.00075
dense_size = 2048
base_path = 'C:/Data/Workspace/GitHub/DeepLearningProject/LocalContent/Logs/'
custom_path = 'Run_v212_400_v1_TestRegFrom_v4'
filename = 'Par_T12000_V1000_D2048_L229_B40_Drop0_4_KReg0_2_Lr0_01_minLr1e-09.E28-L0.216-A0.957'
# Create path
main_path = base_path + custom_path

In [8]:
# Configure the loading of data
# Image Generator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=180,
    shear_range=0.2,
    zoom_range = 0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True)

train_generator = train_datagen.flow_from_directory(
        "../../LocalContent/Datasets/Resized_400/train",  # this is the target directory
        batch_size=batch_size,
        class_mode='categorical') 

# Configure model
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a dropout layer
x = Dropout(rate = dropout)(x)
# let's add a fully-connected layer
x = Dense(dense_size, activation='relu',kernel_regularizer=regularizers.l2(kernel_reg))(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Load possible weights for the model
model.load_weights(main_path + '/' + filename +  '.hdf5')
#model.load_weights('drive/' + filename +  '.hdf5')

Found 3783 images belonging to 12 classes.


In [13]:
# FUNCTION TO PREDICT TEST DATA AND GENERATING A RESULTING CSV FILE FOR KAGGLE
# Read in the test data
source ="../../LocalContent/Datasets/Resized_400/test"
dest = main_path + '/' + filename + '.csv'
#dest = 'drive/' + filename + '.csv'
test_lbl = os.listdir(source)

x_test=[]

# Read the images one by one
for i in test_lbl:
  x = load_img(source + '/'+ i)
  x = img_to_array(x)
  x_test.append(x)
  
# Convert to numpy array  
x_test = np.array(x_test)

# Apply inceptionV3 preprocessing
x_test_std = preprocess_input(x_test)

# Extract class labels from the training generator
class_lbl = list((train_generator.class_indices.keys()))

# Do the prediction
y_predict = model.predict(x_test_std, batch_size = batch_size)
y_predict = y_predict.argmax(axis=-1)

# Write csv file 
f = open(dest, 'wt')
try:
    writer = csv.writer(f)
    writer.writerow( ('file', 'species') )
    for i in range(0, len(y_predict)):
        writer.writerow( (test_lbl[i], class_lbl[y_predict[i]]) )
finally:
    f.close()